In [8]:
from pydantic import BaseModel, Field
from typing import Annotated, Optional, Union, Literal
from langgraph.graph import MessagesState, StateGraph, END
from agent.llm_provider import get_llm_structured, get_llm
from agent.tools.retrieve_pg_tools import vector_store
from langgraph.constants import Send
from langgraph.types import Command
from langchain_core.messages import ToolMessage, SystemMessage, AIMessage, HumanMessage
from operator import add
from pydantic import BaseModel, Field
from typing import Literal
from typing import Optional, Literal, List, Union, Dict, get_args
from pydantic import BaseModel, Field
from typing_extensions import TypedDict, Annotated
from langchain_core.messages import AIMessage, SystemMessage, HumanMessage, ToolMessage
from langgraph.graph import START, END, StateGraph
from langgraph.prebuilt import InjectedState
from langchain_core.tools.base import InjectedToolCallId
from langchain_core.tools import tool
from langgraph.types import Command
from agent.llm_provider import get_llm_structured, get_llm
from agent.tools.retrieve_pg_tools import vector_store
from langgraph.constants import Send
from typing import Optional, Literal, List, Union, Dict, get_args
import os 
import json

class AgentState(MessagesState):
    sender: str
    cv: Optional[str] 
    jds: Annotated[list, add]
    sender: Optional[str]
    new_cv: Optional[str]
    chat_history_summary: str 
    last_index: int = 0
    jd: Optional[str] 
    extractor_insights: Optional[dict] 
    analyst_insights: Optional[dict] 
    suggestor_insights: Optional[dict] 
    goto: str
    content_reviewer_insights: str | dict 
    format_reviewer_insights: str | dict

In [2]:
COORDINATOR_SYSTEM_PROMPT = """
You are CareerFlow, an intelligent AI coordinator that helps users navigate their career journey. 
You specialize in handling greetings and small talk, while routing all other tasks to the correct next step 
using the `next_step` field (R). 

# 🎯 Your Responsibilities
- Introduce yourself as CareerFlow when appropriate
- Respond to greetings (e.g., “hello”, “hi”, “good morning”)
- Engage in small talk (e.g., “how are you”)
- Ask for clarification if the user’s request is unclear
- Route all task-specific or complex inputs to the proper next agent using the `next_step` field
- Always respond in the same language as the user

# 🧭 Request Classification

## 1. Handle Directly
Use `next_step = "__end__"` when:
- The input is a simple greeting or small talk
  e.g. “hi”, “hello”, “how are you?”, “what’s your name?”
- The user asks what you can do or your role
- You need to ask a clarifying question before proceeding

## 2. Route to Next Agent
Use `next_step = "<AGENT>"` when:
- The input contains a specific task or request that matches an agent’s domain:
  
| Request Type                                | Set `next_step`          |
|--------------------------------------------|---------------------------|
| Searching for jobs                         | "job_searcher_agent"      |
| Scoring CV against job descriptions        | "jd_agent"                |
| Ranking or comparing job descriptions      | "jd_agent"                |
| Synthesizing job insights or market trends | "jd_agent"                |
| Reviewing, editing, or aligning a CV       | "cv_agent"                |

→ If the user already provides job IDs (e.g., "score jobs 1, 3, and 5"), you can route directly to `jd_agent` without calling `job_searcher_agent`.
→ If the user only provides a general field (e.g., “marketing”) and no job data is available yet, suggest using `job_searcher_agent` to retrieve relevant jobs first.

📌 **Important**: When setting `next_step` to a specific agent, always include a `message_to_next_agent` that summarizes the user's intent or provides context for the next agent. This ensures a smooth handoff and better user experience.

Example:
- User: "Can you help align my CV to jobs 2, 3, and 5?"
- → `next_step = "jd_agent"`
- → `message_to_next_agent = "The user wants to score their CV against jobs 2, 3, and 5."`

## 3. Not Clear?
- If user input is vague or incomplete (e.g., “Can you help?”), ask a clarifying question
- Set `next_step = "__end__"` until you receive clearer input

# ⚙️ Execution Rules
- Never attempt to perform the task yourself (e.g., scoring, rewriting CVs)
- Only classify the request and assign the appropriate agent
- Never fabricate job listings or data
- Respond clearly, concisely, and kindly

# 📝 Notes
- Do not respond to research or technical questions outside career scope (e.g., LLM theory, system internals)
- Use a helpful and respectful tone
- If unsure, prefer asking the user for clarification
"""

class CoordinatorOutput(BaseModel):
    next_step: Literal['__end__', 'job_searcher_agent', 'cv_agent', 'jd_agent'] = Field(
        default='__end__',
        description=(
            "Next step in the graph flow. Determines which specialized agent will handle the request:\n"
            "- '__end__': End the interaction or continue small talk with CareerFlow.\n"
            "- 'job_searcher_agent': Hand off to Job Search Agent (e.g., search, filter, role exploration).\n"
            "- 'cv_agent': Hand off to CV Agent (e.g., CV review, job alignment, rewrite suggestions).\n"
            "- 'jd_agent': Hand off to JD Agent (e.g., scoring JD relevance, ranking jobs, synthesizing job trends)."
        )
    )
    message_to_user: str = Field(..., description="A friendly message to send to the user.")
    message_to_next_agent: str = Field(..., description=(
            "A summary or instruction to pass to the next agent. "
            "This should clearly explain the user's intent, request, or any necessary context. "
            "Leave empty if `next_step` is '__end__'."
        )
    )
       
def coordinator_node(state: AgentState) -> Command:
    print('state:------', state)
    if isinstance(state['messages'][-1], AIMessage):
        return 
    # Command(update={"messages": [ToolMessage(state['messages'][-1].content, tool_call_id=tool_call_id)]})
    llm = get_llm_structured(CoordinatorOutput)
    messages = state['messages']
    
    response = llm.invoke(
        [SystemMessage(COORDINATOR_SYSTEM_PROMPT)] + messages
    )
    print('res:------', response)
    
    if response.next_step == "__end__":
        # return Command(
        #     goto = response.next_step,
        #     update=,'sender': 'coordinator'},
        # )
        return {"messages": [AIMessage(response.message_to_user)]}
    else:
        return Send(response.next_step, {"messages":  response.message_to_next_agent+ "(user said: "+ messages[-1].content+ ' /no_think', 
                                'sender': 'coordinator',
                                'cv': state.get('cv', ''),
                                'content_reviewer_insights': state.get('content_reviewer_insights', ''),
                                'format_reviewer_insights': state.get('format_reviewer_insights', ''),
                                })

In [3]:
class CVExpertOutput(BaseModel):
    # Định tuyến tác vụ đến agent xử lý hình thức hoặc nội dung CV
    next_step: Literal['cv_format', 'cv_content'] = Field(
        default='cv_format',
        description=(
            "Determines the next specialized agent to handle the request:\n"
            "- 'cv_format': handles formatting and presentation of the CV (by default if user not mention anything)\n"
            "- 'cv_content': handles content relevance and alignment with job description"
        )
    )

    # Loại yêu cầu: đánh giá hay chỉnh sửa CV
    action_type: Literal['review', 'rewrite'] = Field(..., description="Indicates whether the request is for 'review' (assessment only) or 'rewrite' (content update/improvement)")

    # Chỉ mục (index) của job để dùng làm đối chiếu với CV
    jd_index: str = Field(default='4943', description="Index or ID of the Job Description to compare against when analyzing CV content")

CV_SYSTEM_PROMPT = """
You are CVJDCritique, an expert assistant specializing in understanding requests related to CV and job descriptions. 
Your role is to decide the proper next step for processing a CV request, based on the user input and available job descriptions.

# Goals

Your main responsibilities are:
- Identify whether the request is about **formatting** (presentation, layout, style) or **content** (alignment with job descriptions, relevance).
- Determine whether the user wants to **review** (assess only) or **rewrite** (actively edit/improve) the CV.
- Select the correct job description from a list of indexed JDs (e.g., 4942, 7383) to compare against.
- If the JD is not provided in the current state, use the `job_index` to fetch the correct one later.

# Request Classification

1. **CV Formatting (`next_step = "cv_format"`)**:
    - The user talks about layout, design, style, visual appeal, or professionalism.
    - Examples:
        - “Can you check if my CV looks clean and professional?”
        - “I want help formatting my resume.”

2. **CV Content (`next_step = "cv_content"`)**:
    - The user mentions relevance, matching, alignment with roles, skills, or qualifications.
    - Examples:
        - “Is my CV a good fit for this data analyst job?”
        - “Does my resume align with the JD?”

3. **Review (`action_type = "review"`)**:
    - The user asks to evaluate, assess, or give feedback without rewriting.
    - Examples:
        - “Can you check if my CV is okay?”
        - “Please review my resume.”

4. **Rewrite (`action_type = "rewrite"`)**:
    - The user explicitly asks for rewriting, improving, or editing the CV.
    - Examples:
        - “Can you help me rewrite this for a better fit?”
        - “Please improve my CV for this job.”

# Job Index

- If the user refers to a specific job, role, or context, infer the correct `job_index` (e.g., "4942").
- You must provide a valid `job_index` so that the correct JD can be retrieved later.

# Output Format

You must produce a JSON output with:
- `next_step`: either `"cv_format"` or `"cv_content"`
- `action_type`: either `"review"` or `"rewrite"`
- `jd_index`: a string identifier of the job description to be used for comparison

# Notes

- Keep your reasoning internal — only return the structured output.
- Always make sure all three fields are present in your response.
- Never assume JD content is available unless specified; always include `jd_index`.

"""
def cv_expert(state) -> Command:
    print("--cv-jd--")
    print("----",type(state))
    jd = state.get('jd', '')
    
    if not state.get('cv', ''):
        return Command(
            goto = state['sender'],
            update={"messages": [AIMessage('CV is not uploaded yet')],'sender': 'coordinator'},
        )
            
    llm = get_llm_structured(CVExpertOutput)
    response = llm.invoke([SystemMessage(CV_SYSTEM_PROMPT)] + state["messages"])
    print('response: ', response)
    
    if response.next_step == 'cv_content':
        if not jd:
            try:
                jd = vector_store.get_by_ids([response.jd_index])[0].page_content
            except Exception as e:
                print(e)
                return Command(
                    goto = state['sender'],
                    update={"messages": [AIMessage('Job index is not available try default value 4943')],'sender': 'cv_jd_agent'},
                )
        feedback =     'content_reviewer_insights'
    else:
        feedback =     'format_reviewer_insights'
        
    print('----', feedback, '----')
    if response.action_type == 'rewrite' and state.get(feedback, ''):   
        print('da danh gia truoc roi, chuyen qua viet luon') 
        return Send('cv_writer', {'sender': feedback,
                                    'goto': response.action_type,
                                    'jd': jd,
                                    'cv': state['cv'],
                                    feedback: state.get(feedback, '')
                                    })
    elif response.action_type == 'review' and state.get(feedback, ''):   
        print('da danh gia truoc roi, chuyen qua viet luon') 
        return Send('cv_writer', {'sender': feedback,
                                    'goto': response.action_type,
                                    'jd': jd,
                                    'cv': state['cv'],
                                    feedback: state.get(feedback, '')
                                    })
    else:
        if response.next_step == 'cv_content':
            return Send('jd_extractor', {'sender': 'cv_expert','goto': response.action_type,
                                    'jd': jd, 'cv': state['cv']})

        return Send(response.next_step, {'sender': 'cv_expert','goto': response.action_type,
                                    'jd': jd, 'cv': state['cv']})
                
        

In [4]:
def format_reviewer(state: AgentState) -> Command:
    
    FORMAT_REVIEW_SYSTEM_PROMPT = """
You are an HR expert whose task is to review and evaluate a candidate’s CV based on the following detailed criteria. Carefully read and analyze the CV, then provide clear, actionable feedback. Highlight specific errors and offer direct suggestions for improvement. Avoid vague or generic comments. Follow these evaluation points:
	1.	Relevance to the job:

	•	Does the CV show a clear connection to the target job?
	•	Are relevant skills, education, and keywords included?

	2.	Timeline structure:

	•	Are all time periods for education and work fully listed?
	•	Is the timeline sorted in reverse chronological order?
	•	Are there unexplained time gaps?

	3.	Personal information:

	•	Is the candidate’s name clearly and prominently displayed?
	•	Is the email professional (ideally containing the name)?
	•	Is the phone number formatted for easy reading (e.g., 0123 456 789)?
	•	Are social media links appropriate and professional (prefer LinkedIn over Facebook)?
	•	Is the address overly detailed? (Only district/city is needed)
	•	Are there unnecessary details like ID number, gender, age, or marital status?
	•	Is the personal info section concise (less than 4 lines)?
	•	Is the photo professional (no selfies, clear face, plain background)?

	4.	Career objective (optional):

	•	Is the career objective clear and concise (maximum 4 lines)?
	•	Is it personalized, specific, and not generic?

	5.	Education:

	•	Does it include institution name, major, and start/end dates?
	•	Is the order correct (most recent first)?
	•	Are relevant courses or certifications beyond formal degrees listed?

	6.	Skills:

	•	Is there a skills section?
	•	Are the skills relevant to the job?
	•	Are soft skills and technical skills clearly differentiated?
	•	Are rating bars used without clear criteria? If yes, remove or replace with measurable standards (e.g., IELTS 6.5).
	•	Are there 4–8 skills listed? Too few or too many may be ineffective.
	•	Are action verbs used to describe the skills? (e.g., “Manage a team” instead of “Leadership”)

	7.	Work experience & activities:

	•	Is the timeline sorted from most recent to oldest?
	•	Are there unexplained gaps?
	•	Does each job include a short description of the company (size, product, industry)?
	•	Are there measurable results (numbers, KPIs, achievements)?
	•	Does each role have at least 3 bullet points describing tasks and outcomes?

	8.	References:

	•	Are references only included when requested?
	•	If listed, do they contain full name, position, company, email, and phone number?

	9.	Hobbies:

	•	Do the listed hobbies help the candidate stand out?
	•	Are they specific (e.g., reading psychology books, listening to classical music)?

	10.	Overall layout and formatting:

	•	Is the structure logical? (Personal Info > Skills > Experience > Education)
	•	Is the length appropriate (between 40–80 lines)?
	•	Are personal pronouns like “I”, “My”, or “Me” avoided?

After reviewing, return your feedback in the form of a checklist or structured list, pointing out both the problems and suggested improvements.
"""
    class Feedback(BaseModel):
        issue: str = Field(description="The issue identified by the human reviewer")
        solution: str = Field(description="The solution to the issue identified by the human reviewer")
        criteria: str = Field(description="The criteria used to evaluate the CV")
            
    class Feedbacks(BaseModel):
        feedbacks: list[Feedback] = Field(description="The feedbacks from the human reviewers")
    
    print('--review format--')
    if state['goto'] == 'review':
        # tra loi truc tiep
        llm = get_llm()
    else:
        llm = get_llm_structured(Feedbacks)
        
    response = llm.invoke([SystemMessage(FORMAT_REVIEW_SYSTEM_PROMPT), HumanMessage(f'Start review {state["cv"]} /no_think')])
    
    print('--done review format--')
    
    
    if state['goto'] == 'review':
        # end process
        return Command(
            goto = 'coordinator',
            graph = Command.PARENT,
            update={"messages": [AIMessage(response.content)],'sender': 'format_reviewer',
                    "format_reviewer_insights":response.content},
        )
    else:
        return Command(
            goto = 'cv_writer',
            update={'sender': 'format_reviewer', "format_reviewer_insights":response},
        )

In [5]:
def jd_extractor(state):
    EXTRACTOR_INSTRUCTION = """
# Role and Objective  
You are a Job Description (JD) analysis expert. Your mission is to deeply analyze the provided JD and extract key hiring signals that reveal the employer’s true expectations and priorities for the ideal candidate. You are not just listing what’s mentioned — you must infer deeper intent and structure your output clearly.

# Task  
Analyze the given job description and extract the most important requirements and signals. Your output must be paraphrased and organized into **five categories**:

## Output Format (Use exact structure):
1. **Technical Skills**  
List relevant tools, technologies, programming languages, frameworks, platforms, or systems explicitly or implicitly required.

2. **Soft Skills**  
List essential non-technical traits or behaviors (e.g. communication, teamwork, leadership, problem-solving).

3. **Experience Requirements**  
Summarize the years of experience, industry background, project types, or job roles needed.

4. **Education & Certifications**  
List any degrees, majors, educational levels, or certifications that are required or preferred.

5. **Hidden Insights (Recruiter Intent)**  
Reflect on the overall tone, emphasis, and structure of the JD to infer the *real* goals behind the role.  
- Why is this role open now?  
- What kind of personality or archetype fits best?  
- What challenges might this role be solving?

# Instructions  

- **Paraphrase only** — Do **not** copy-paste content from the JD.  
- **Think critically** — Focus on what matters most, not just what’s explicitly stated.  
- **Be concise** — Use clean, bullet-pointed lists that are easy to scan.  
- **Infer meaning** — Capture both overt and subtle signals from the JD.  

# Reasoning Steps  
1. Read the full JD carefully.  
2. Identify explicit requirements and implied expectations.  
3. Categorize the findings under the 5 output sections.  
4. Paraphrase everything professionally and clearly.  

Final Note

You are acting as a specialist whose job is to extract and interpret hiring intent behind job postings. Respond with only the structured output. Do not include any explanations, introductions, or summaries outside of the 5 required categories.
"""
    class ExtractorOutput(BaseModel):
        technical_skills: List[str] = Field(
            ..., 
            description="List of tools, technologies, programming languages, frameworks, platforms, or systems required or implied."
        )
        soft_skills: List[str] = Field(
            ..., 
            description="List of essential non-technical traits or behaviors expected for the role."
        )
        experience_requirements: List[str] = Field(
            ..., 
            description="Summarized expectations for years of experience, industries, project types, or job roles."
        )
        education_certifications: List[str] = Field(
            ..., 
            description="Degrees, majors, education levels, or certifications that are required or preferred."
        )
        hidden_insights: List[str] = Field(
            ..., 
            description="Inferred recruiter intent such as purpose of the role, ideal personality, timing, or underlying organizational needs."
        )
    
    print('--extract--')
    print(type(state), state.keys())
    
    
    structured_llm = get_llm_structured(ExtractorOutput)
    response = structured_llm.invoke(
        [SystemMessage(EXTRACTOR_INSTRUCTION), HumanMessage(f'Start extracting {state["jd"]}')]
    )
    print("--done extract--")
    # return Send('analyze_cv', {"extractor_insights": response, "curriculum_vitae": state["curriculum_vitae"]})
    return {"extractor_insights": response}
        
def cv_analyst(state):
    ANALYST_INSTRUCTION = """# Role and Objective
You are a career analysis expert. Your mission is to evaluate how well a candidate's CV aligns with the key hiring criteria extracted from a job description (JD).

You will be given:
- A list of **JD Insights** (divided into 5 categories).
- The candidate’s **CV** (in text form).

# Task
For each of the 5 JD insight categories listed below, assess the CV and provide structured feedback.

## Output Format (Use exact structure):

Return a list of 5 feedback items, each containing:

1. **name**  
One of the following exact values:
   - `technical_skills`
   - `soft_skills`
   - `experience_requirements`
   - `education_certifications`
   - `hidden_insights`

2. **score**  
Rate from 0 to 10 how well the CV fulfills the expectations for that category (higher is better).

3. **comment**  
Give a concise, professional evaluation. Explain how well the CV meets this requirement. Mention any gaps or strong points.

## Scoring Guide
- **9–10**: Fully satisfies the requirement; matches perfectly or exceeds expectations.
- **7–8**: Mostly meets expectations; minor gaps.
- **4–6**: Partially meets expectations; some important aspects missing.
- **1–3**: Barely relevant; major issues.
- **0**: Not addressed at all in the CV.

# Categories to Evaluate
You must review the CV according to the following categories:

1. **Technical Skills**  
Compare listed tools, technologies, platforms, or programming languages with what the JD expects.

2. **Soft Skills**  
Evaluate evidence of interpersonal traits like communication, teamwork, leadership, adaptability, etc.

3. **Experience Requirements**  
Check for years of experience, relevant industries, types of projects, and similar past roles.

4. **Education & Certifications**  
Look for relevant degrees, fields of study, or professional certifications.

5. **Hidden Insights (Recruiter Intent)**  
Assess whether the candidate seems to match the deeper intent behind the role (e.g. personality, purpose of the role, cultural fit).

# Instructions
- Be objective and professional.
- Justify the score with a short but insightful comment.
- Think critically. Don’t just match keywords — understand fit.
- Respond ONLY with the structured output, nothing more.

# Input Context

## JD Insights
{insights}
"""
    class Feedback(BaseModel):
        name: Literal[
            'technical_skills',
            'soft_skills',
            'experience_requirements',
            'education_certifications',
            'hidden_insights'
        ] = Field(..., description="The criteria being evaluated.")
        score: int = Field(..., ge=0, le=10, description="Score from 0 to 10 reflecting how well the CV meets this requirement.")
        comment: str = Field(..., description="Brief comment analyzing how well the CV fulfills this requirement.")

    class AnalystOutput(BaseModel):
        feedbacks: List[Feedback] = Field(..., description="List of feedback items for each key requirement category.")
    print("--analyze--")
    print(type(state), state.keys())

    system_message = ANALYST_INSTRUCTION.format(
        insights = state["extractor_insights"]
    )

    structured_llm = get_llm_structured(AnalystOutput)
    response = structured_llm.invoke(
        [SystemMessage(system_message), HumanMessage(f'Start analyzing cv: {state["cv"]}')]
    )
    print("--done analyze--")

    # return Send('suggest_cv', {"insights": response, "curriculum_vitae": state["curriculum_vitae"]})
    return {"analyst_insights": response}
        


def content_reviewer(state: AgentState):
    
    SUGGESTOR_INSTRUCTION = """# Role and Objective
You are a professional CV improvement advisor. Your task is to review structured feedback about how well a CV matches a job description, and determine whether and how the CV can be improved.

Your job is to:
- Assess each requirement category based on the feedback
- Decide whether improvements are possible through rewriting or restructuring the CV
- Suggest how to improve it if possible
- Identify relevant keywords or phrases that could be added to enhance alignment with the JD

# Input
You will receive a list of 5 feedback entries. Each entry includes:
- `name`: the requirement category (e.g., "technical_skills")
- `score`: numeric score (0–10) reflecting how well the CV meets that requirement
- `comment`: a brief explanation of the score

# For each category, return the following:
1. `name`: the category name (keep unchanged)
2. `action_needed`: one of:
   - `"yes"` → Improvements are possible and recommended
   - `"no"` → Already strong, no change needed
   - `"cannot_be_improved"` → Gaps reflect real limitations that cannot be fixed through CV editing
3. `recommendation`: 
   - If `"yes"` → Suggest what to rephrase, highlight, or clarify in the CV
   - If `"no"` → Briefly note the strength
   - If `"cannot_be_improved"` → Explain why improvement is not possible through writing
4. `suggested_keywords`: 
   - A list of relevant terms (technologies, behaviors, job titles, etc.) that could be added to improve alignment
   - Leave empty (`[]`) if `action_needed` is `"no"` or `"cannot_be_improved"`

# Guidelines
- Be realistic: do not suggest faking credentials or experience.
- Think critically about what can actually be added, emphasized, or reworded in a CV.
- Use bullet points and keep output concise and professional.

# Input Context
**Feedback List:**
{insights}"""
    
    class ImprovementSuggestion(BaseModel):
        name: Literal[
            'technical_skills',
            'soft_skills',
            'experience_requirements',
            'education_certifications',
            'hidden_insights'
        ] = Field(..., description="The requirement category being evaluated.")
        
        action_needed: Literal[
            'yes',                 # CV can and should be improved for this aspect
            'no',                  # This aspect is already strong, no changes needed
            'cannot_be_improved'   # This gap reflects something that cannot realistically be improved through CV edits (e.g., lack of years of experience)
        ] = Field(..., description="Whether the CV can be improved in this area.")
        
        current_expression: str = Field(..., description="How the candidate has currently demonstrated or expressed this aspect in the CV. Paraphrase or quote relevant parts.")
        
        recommendation: str = Field(..., description="Suggestion on what to change, emphasize, or reword in the CV. If no action is needed or not possible, explain why.")
        
        suggested_keywords: List[str] = Field(..., description="List of relevant keywords (technologies, soft skills, etc.) to consider adding to improve alignment with the JD.")


    class SuggestorOutput(BaseModel):
        suggestions: List[ImprovementSuggestion] = Field(..., description="List of improvement suggestions for each requirement category.")


    print('--review content--')
    print(type(state), state.keys())
    

    system_message = SUGGESTOR_INSTRUCTION.format(
        insights = state["analyst_insights"]
    )
    if state['goto'] == 'review':
        llm = get_llm()
    
    else:
        llm = get_llm_structured(SuggestorOutput)
    response = llm.invoke(
        [SystemMessage(system_message), HumanMessage(f"Start reviewing cv: {state['cv']}")]
    )
    print('--done review content--')
    
    if state['goto'] == 'review':
        
        return Command(
            goto = 'coordinator',
            graph = Command.PARENT,
            
            update={"messages": [AIMessage(response.content)],'sender': 'content_reviewer',
                    "content_reviewer_insights": response},
        )
    else:
        return Command(
            goto = 'cv_writer',
            update={'sender': 'content_reviewer', "content_reviewer_insights": response},
        )
        
# workflow = StateGraph(AgentState)
# workflow.add_node("jd_extractor", jd_extractor)
# workflow.add_node("cv_analyst", cv_analyst)
# workflow.add_node("content_reviewer", content_reviewer)
# workflow.set_entry_point("jd_extractor")
# workflow.add_edge('jd_extractor',"cv_analyst")
# workflow.add_edge('cv_analyst',"content_reviewer")
# # workflow.set_finish_point("content_reviewer")
# ContentReviewer = workflow.compile()

In [6]:
def cv_writer(state):
    
    WRITER_INSTRUCTION = """
# Role and Objective  
You are a professional CV editing assistant. Your job is to improve the candidate’s CV to better align with a job description (JD), based on a structured improvement plan generated by a JD-CV analysis agent.

Your edits should make the CV more relevant, professional, and compelling — while staying truthful to the candidate’s background.

# Task  
Use the improvement suggestions provided to revise the CV. You may:
- Emphasize or reword existing experience
- Add missing but plausible details based on the candidate's background
- Inject relevant keywords or terminology from the job description
- Reorganize or clarify content for better readability

# Editing Rules  
- **Do not fabricate** experience that doesn’t exist (e.g., fake years of experience or degrees).
- You may reframe or emphasize relevant work to match JD expectations.
- Do not add certifications, education, or job titles the candidate does not have.
- Add relevant **keywords** where appropriate, based on suggestion.
- Be clear, concise, and use a professional tone.
- You can update or add bullet points, rephrase summaries, and enhance skill sections.

# Input  
You will receive the following:
1. The original CV text.
2. A list of improvement suggestions in structured format, each with:
   - The category (e.g. technical skills, soft skills)
   - Whether an action is needed
   - A recommendation
   - Suggested keywords

# Output  
Return the **revised CV text only** — do not include any commentary or explanation. The CV should reflect the edits and improvements based on the provided plan.

Improvement Plan:

{insights}

Final Note

Do not mention that this is an edited CV. Just return the improved CV text.
"""


    class WriterOutput(BaseModel):
        new_cv: str = Field(..., description="The fully rewritten and improved CV based on the provided suggestions. The output should be clean, professional, and align closely with the job description.")

        
    print('--writer--')
    print(type(state), state)
    curriculum_vitae = state["cv"]
    
    if state['sender'].startswith('content_reviewer'):
        insights = state.get('content_reviewer_insights', '')
    elif state['sender'].startswith('format_reviewer'):
        insights = state.get('format_reviewer_insights', '')
    else:
        insights = ''

    system_message = WRITER_INSTRUCTION.format(
        insights = insights
    )
    llm = get_llm()
    response = llm.invoke(
        [SystemMessage(system_message), HumanMessage(f'Start rewriting cv: {curriculum_vitae} /no_think')]
    )
    # extractor = get_llm_structured(WriterOutput)
    # final = extractor.invoke([HumanMessage(f"""The following message contains a rewritten CV. Please extract **only the full CV text** from it.
    # Message:
    # {response.content}""")])
    
    print('--done write--')
    
    return Command(
        goto = 'coordinator',
        graph = Command.PARENT,
        update={'sender': 'writer', "messages": response, 'new_cv': response},
    )


In [7]:
workflow = StateGraph(AgentState)
workflow.add_node("cv_jd_expert", cv_expert)

workflow.add_node("cv_format", format_reviewer)

workflow.add_node("jd_extractor", jd_extractor)
workflow.add_node("cv_analyst", cv_analyst)
workflow.add_node("content_reviewer", content_reviewer)
workflow.add_edge('jd_extractor',"cv_analyst")
workflow.add_edge('cv_analyst',"content_reviewer")

workflow.add_node("cv_writer", cv_writer)

workflow.set_entry_point("cv_jd_expert")
# workflow.set_finish_point("cv_writer")

In [8]:
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()
CVExpert = workflow.compile(checkpointer=memory)

In [9]:
# CVJDExpert.invoke({'messages': [HumanMessage('jd 1337')], 'cv': 'Hi im anhduc'},{"configurable": {"thread_id": "1"}})


In [10]:

from agent.tools import job_search_by_cv as search_by_cv
from agent.tools import job_search_by_query as search_by_keyword
from langgraph.prebuilt import tools_condition, ToolNode



JOB_SEARCHER_SYSTEM_PROMPT = """
You are an expert job search assistant. Your job is to help users find relevant job postings either by using specific keywords or by analyzing their CV.

Details

Your primary responsibilities are:
	•	Determining whether to use search_by_keyword() or search_by_cv() based on the user input
	•	Asking the user for clarification or input if needed
	•	Calling the appropriate tool and letting it return the results
	•	Always responding in the same language as the user
	•	You must not attempt to generate or fabricate job listings yourself
	•	You should keep your response brief, friendly, and informative after the tool runs

Execution Rules
	•	Only call one of the two tools depending on the request
	•	After the tool executes, provide a clear and friendly message with the results

Notes
	•	Keep responses concise and focused
	•	Your only goal is to route the request to search_by_keyword() or search_by_cv() appropriately
	•	You should not handle any career advice, market analysis, or CV review — those belong to other agents
	•	When in doubt, ask for more details rather than guessing
"""


def job_agent_node(state: AgentState):
    print("--job searcher--")
    print("--state: ", state['messages'])
    llm = get_llm().bind_tools([search_by_cv, search_by_keyword], )
    messages = state["messages"]
    
    if isinstance(messages[-1], ToolMessage):
        messages.append(HumanMessage('/no_think'))
    
    response = llm.invoke([SystemMessage(JOB_SEARCHER_SYSTEM_PROMPT)] + messages)
    return {"messages": [response], 'sender': state['sender']}
	# return Command(
	# 	# goto = sender,
	# 	update={'sender': 'job_seacher'},
	# 	# update={"messages": [AIMessage(response.content)],'sender': 'job_agent'},
	# )
def router(state):
    print("--router--")
    print("----",state['sender'])
    sender = state['sender']
    return Command(
		goto = sender,
  		graph=Command.PARENT,
		update={'sender': 'job_seacher',"messages": [state["messages"][-1]]},)


builder = StateGraph(AgentState)
builder.add_node("assistant", job_agent_node)
builder.add_node("router", router)

builder.add_node("tools", ToolNode([search_by_cv, search_by_keyword]))
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    tools_condition,
    {'tools': 'tools',END: "router" }
)
builder.add_edge("tools", "assistant")
job_seacher = builder.compile()

In [11]:
# workflow = StateGraph(AgentState)
# workflow.add_node("coordinator", coordinator_node)
# workflow.add_node("job_searcher_agent", job_seacher)
# workflow.add_node("router", router)
# workflow.add_node("cv_jd_agent", CVJDExpert)


# workflow.set_entry_point("coordinator")
# workflow.add_edge('job_searcher_agent',"router")

# from langgraph.checkpoint.memory import MemorySaver
# memory = MemorySaver()
# career_agent = workflow.compile(checkpointer=memory)


# # career_agent.invoke({'messages': [HumanMessage('find me 3 job data analyst')]},{"configurable": {"thread_id": "1"}})

In [13]:
# state= {'messages': [HumanMessage('review my cv format')], 'cv': CV_CONTENT}
# config = {"configurable": {"thread_id": "1"}}
# for i, (msg, metadata) in enumerate(career_agent.stream(state, config, stream_mode="messages")):
#     print(metadata["langgraph_node"], i, msg.content, )

In [14]:
state= {'messages': [HumanMessage('align/rewrite my cv format')], 'cv': CV_CONTENT}
config = {"configurable": {"thread_id": "1"}}
# for i, (msg, metadata) in enumerate(career_agent.stream(state, config, stream_mode="messages")):
#     print(metadata["langgraph_node"], i, msg.content, )

In [15]:
SYNTHESIZE_SYSTEM_PROMPT = """You are an AI assistant helping users analyze job descriptions.

Your task is to extract and structure the main components of the job from the following description:
Extract using the structured format.
"""
class JobCriteriaComparison(BaseModel):
    job_responsibilities: str = Field(..., description="Key responsibilities listed in the job")
    technical_skills_tools: str = Field(..., description="Required technical skills or tools")
    years_of_experience: str = Field(..., description="Years of experience required")
    education_certifications: str = Field(..., description="Required education or certifications")
    soft_skills: str = Field(..., description="Required soft skills or personality traits")
    industry_sector: str = Field(..., description="Industry or sector the job belongs to")
    location_mode: str = Field(..., description="Location and work mode: Remote / Hybrid / On-site")
    salary_range: str | None = Field(None, description="Salary range if mentioned")
    career_growth: str | None = Field(None, description="Mention of career growth or advancement opportunities")
    unique_aspects: str | None = Field(None, description="Any unique benefits or characteristics of the job")

class AnalyzeState(MessagesState):
    jd: str
    jds: List[str]
    jd_analysis: Annotated[list, add]
    jd_indices: list
    summary: str

# ---------------------------- AGENT LOGIC ----------------------------
def get_jd(state):
    print('--get_jd--', state)
    jds = vector_store.get_by_ids([str(i) for i in state["jd_indices"]])
    jds = [jd.page_content for jd in jds]
    
    return {"jds": jds}

def router(state):
    """Route each JD into the extraction node"""
    print('--router--', state)
    
    if state.get("jds", []):
        return [Send("extract", {"jd": jd}) for jd in state['jds']]
    else:
        return Command(goto = 'assistant',
        graph = Command.PARENT,
                   
        update = {"messages": [AIMessage('fail to route')]})

def parser_agent(state): 
    print('--parser--')
    
    jd = state.get("jd", "")
    llm = get_llm_structured(JobCriteriaComparison)
    response = llm.invoke([
        SystemMessage(SYNTHESIZE_SYSTEM_PROMPT),
        HumanMessage(f"Conduct extraction this jd :{jd}")
    ])
    
    return Command(update = {"jd_analysis": [response]})

SUMMARIZE_SYSTEM_PROMPT = """You are a hiring analyst AI assistant. Your task is to summarize and synthesize multiple job descriptions (JDs) that share the same job title.

Each JD has been analyzed based on a set of common criteria such as responsibilities, required skills, experience, education, and soft skills.

Your goal is to:
1. Identify common patterns across the JDs.
2. Highlight differences or variations.
3. Note any unique features in any JD.
4. Optionally, categorize JDs into types.
5. Provide a final summary insight about the market for this role.

Use markdown formatting and bullet points/tables if appropriate.
"""
def summarize_agent(state):
    print('--summary--')
    jd_analysis = state['jd_analysis']
    llm = get_llm()
    response = llm.invoke([
        SystemMessage(SUMMARIZE_SYSTEM_PROMPT),
        HumanMessage(f"Here are the analyses: {jd_analysis}. /no_think")
    ])
    # print("--response--", response)
    return Command(goto = 'assistant',
        graph = Command.PARENT,
                   
        update = {"messages": [response]})

# ---------------------------- GRAPH ----------------------------
graph = StateGraph(AnalyzeState)
graph.add_node("get_jd", get_jd)
graph.add_node("extract", parser_agent)
graph.add_node("summarize", summarize_agent)

graph.set_entry_point("get_jd")
graph.add_conditional_edges("get_jd", router, ["extract"])

graph.add_edge("extract", "summarize")
graph.set_finish_point("summarize")

synthesize_agent = graph.compile()

In [16]:
# synthesize_agent.invoke({'messages': 'hi', "jd_indices": ['1337', '4946']})

In [30]:

# ---------------------------- PROMPT ----------------------------
# Prompt dùng để chấm điểm CV theo từng JD
SCORE_PROMPT_SYSTEM = """
You are an expert in evaluating the relevance between a candidate's CV and a specific Job Description (JD).

Your task is to score how well the candidate matches the job across different criteria, and assign **a weight for each criterion** based on the **importance of that factor in the given JD**.

Please follow the rules below:

1. For each evaluation criterion, give:
   - A score from 0 to 10 based on the candidate's CV.
   - A weight from 0.0 to 1.0 representing how important that criterion is in the JD.

2. Then, write **one brief overall comment** summarizing the candidate’s fit.

3. Do not calculate the final average score. It will be computed automatically.

### Fields to fill:

- `id`: Index or ID of the Job Description being evaluated.
- `job_title_relevance`: Does the candidate's experience match the job title?
- `years_of_experience`: Is the experience duration sufficient?
- `required_skills_match`: Does the candidate possess the required technical skills?
- `education_certification`: Is the academic background suitable?
- `project_work_history`: Are the past projects relevant?
- `softskills_language`: Does the candidate demonstrate useful soft skills?
- `*_weight`: For each of the above criteria, specify how important it is (0.0 to 1.0) for this job.
- `overall_comment`: Give one short paragraph with your summary evaluation.
- Leave `overall_fit_score` as 0; it will be computed after scoring.

### Example:
If the JD is for a senior backend engineer with a focus on Golang microservices, then:
- `required_skills_match_weight` might be 0.9 (very important),
- `education_certification_weight` might be 0.3 (less important).

Return the result strictly in the schema format.
"""

# Prompt dùng để tổng hợp các đánh giá thành 1 đoạn summary
summary_instruction = """You are an AI assistant summarizing HR evaluations.

You are given a list of evaluation objects that scored one CV against multiple job descriptions.

Summarize the following:
- Patterns or trends across evaluations
- Overall candidate fit across all jobs
- Any notable strengths or weaknesses
- If applicable, recommend the best-fitting job(s)

Only refer to information available in the analysis. Be concise and insightful.
"""

# ---------------------------- SCHEMA ----------------------------
from pydantic import BaseModel, Field
from typing import Literal
from pydantic import BaseModel, Field, model_validator

class CVJDMatchFeedback(BaseModel):
    id: int = Field(..., description="Job index")

    job_title_relevance: int = Field(..., ge=0, le=10, description="Score (0-10): How well does the candidate's experience align with the job title?")
    job_title_weight: float = Field(..., ge=0.0, le=1.0, description="Weight (0-1): Importance of job title relevance for this role.")
    
    years_of_experience: int = Field(..., ge=0, le=10, description="Score (0-10): Does the candidate have sufficient experience for the position?")
    years_of_experience_weight: float = Field(..., ge=0.0, le=1.0, description="Weight (0-1): Importance of experience duration.")

    required_skills_match: int = Field(..., ge=0, le=10, description="Score (0-10): To what extent does the candidate possess the skills listed in the JD?")
    required_skills_weight: float = Field(..., ge=0.0, le=1.0, description="Weight (0-1): Importance of matching required skills.")

    education_certification: int = Field(..., ge=0, le=10, description="Score (0-10): Does the candidate's academic background fit the job requirements?")
    education_certification_weight: float = Field(..., ge=0.0, le=1.0, description="Weight (0-1): Importance of educational background.")

    project_work_history: int = Field(..., ge=0, le=10, description="Score (0-10): Are the candidate’s past projects or roles relevant to this position?")
    project_work_history_weight: float = Field(..., ge=0.0, le=1.0, description="Weight (0-1): Importance of work/project experience relevance.")

    softskills_language: int = Field(..., ge=0, le=10, description="Score (0-10): Does the candidate show relevant communication, leadership, or other soft skills?")
    softskills_language_weight: float = Field(..., ge=0.0, le=1.0, description="Weight (0-1): Importance of soft skills and communication.")

    overall_comment: str = Field(..., description="One overall comment about the candidate’s fit for the job.")
    overall_fit_score:       float = Field(0, description="Average score (0-10) calculated from all score fields.")
    
    
    @model_validator(mode="after")
    def compute_overall_score(self):
        self.overall_fit_score = round(
            self.job_title_relevance * self.job_title_weight +
            self.years_of_experience * self.years_of_experience_weight +
            self.required_skills_match * self.required_skills_weight +
            self.education_certification * self.education_certification_weight +
            self.project_work_history * self.project_work_history_weight +
            self.softskills_language * self.softskills_language_weight
        )
        return self

def format_cvjd_feedback_list(feedback_list: list[CVJDMatchFeedback]) -> str:
    output = []
    for fb in feedback_list:
        text = f"""\
        Job Index {fb.id}:
        - Job Title Relevance:       {fb.job_title_relevance}/10
        - Years of Experience:       {fb.years_of_experience}/10
        - Required Skills Match:     {fb.required_skills_match}/10
        - Education & Certification: {fb.education_certification}/10
        - Project Work History:      {fb.project_work_history}/10
        - Soft Skills & Language:    {fb.softskills_language}/10
        - Overall Fit Score:          {fb.overall_fit_score}/10

        Comment: {fb.overall_comment}
        """
        
        output.append(text)
    return "\n".join(output)

# ---------------------------- GRAPH STATE ----------------------------
class ScoreState(MessagesState):
    cv: str
    jd_index: str
    jds: List[str]
    scored_jds: Annotated[list | list[CVJDMatchFeedback], add]
    jd_indices: List[str]
    


# ---------------------------- AGENT LOGIC ----------------------------

def get_jd(state):
    print('--get_jd--', state)
    jds = vector_store.get_by_ids([str(i) for i in state["jd_indices"]])
    jds = [jd.page_content for jd in jds]
    
    return {"jds": jds}

def router(state):
    """Route each JD into the extraction node"""
    print('--router--', state)
    
    if state.get("jds", []):
        return [Send("score", {"jd": jd, "cv": state["cv"], 'jd_index': id}) for jd, id in zip(state["jds"], state["jd_indices"])]
    else:
        return Command(goto = 'assistant',
        graph = Command.PARENT,
                   
        update = {"messages": [AIMessage('fail to route')]})
        
        
def score_agent(state): #: Annotated[ScoreState, InjectedState]):
    print("--score--")
    
    jd = state.get("jd", "")
    cv = state.get("cv", "")
    
    llm = get_llm_structured(CVJDMatchFeedback)
    response = llm.invoke([
        SystemMessage(SCORE_PROMPT_SYSTEM),
        HumanMessage(f"Conduct scoring job {state['jd_index']}: {jd} with cv: {cv} . /no_think")
    ])
    print(type(response), "response from score",  response)
    print("state", state)
    return {"scored_jds": [response]}

def summarize_score_agent(state):
    print("--summa--")
    jd_analysis = state.get("scored_jds", [])
    print(state)
    llm = get_llm()
    response = llm.invoke([
        SystemMessage(summary_instruction),
        HumanMessage(f"Here are the analyses of jobs: {jd_analysis}. /no_think")
    ])
    print(response)
    return Command(goto = 'coordinator',
        graph = Command.PARENT,
                   
        update = {"messages": [response]})

# ---------------------------- GRAPH ----------------------------

def build_score_graph() -> StateGraph:
    score_graph = StateGraph(ScoreState)
    score_graph.add_node("get_jd", get_jd)
    score_graph.add_node("score", score_agent)
    score_graph.add_node("summarize", summarize_score_agent)

    score_graph.set_entry_point("get_jd")
    score_graph.add_conditional_edges("get_jd", router, ["score"])    
    score_graph.add_edge("score", "summarize")
    score_graph.set_finish_point("summarize")

    return score_graph.compile()

score_agent = build_score_graph()

In [31]:
# score_agent.invoke({"jd_indices": ['1337', '4946'], 'cv': 'Anh Duc - Data Analyst'})
#

In [32]:

from agent.tools import job_search_by_cv as search_by_cv
from agent.tools import job_search_by_query as search_by_keyword
from langgraph.prebuilt import tools_condition, ToolNode



JD_SYSTEM_PROMPT = """
You are  a coordination agent designed to route user job-related requests to the correct tools. 
You cannot fabricate data or provide market analysis beyond what is in the job descriptions (JDs). 
Your job is to understand the user's intent and invoke one of the available tools to help them.

How to Act:
1. If the user asks to explore or analyze a field like "marketing" or "UI/UX jobs", call `call_job_seacher` first, alway call job_searcher first.
    • After that, you may use scoring or synthesis tools as appropriate.
    
2. If the user asks how well their CV fits a role or job, and JD indices are already available, use `call_score_jds()`.

3. If the user wants to compare multiple roles or asks something like “What do these jobs have in common?”, use `call_synthesize_jds()`.

4. If a tool cannot be used due to missing information (e.g., no JD indices yet), ask the user to specify the role or interest area.
    • Never fabricate data or assume.

5. If the user asks for market trend analysis, politely explain that you cannot analyze the broader labor market, 
    but can help synthesize existing job descriptions for insights.

Never:
• Make up job descriptions or scores.
• Give CV advice or generate new documents — other agents handle those.
• Attempt to analyze the job market outside of loaded job descriptions.
"""

# class CVJDsOutput(BaseModel):
#     task: Literal['score_jds', 'synthesize_jds', 'call_job_seacher']
#     jd_index: List['str']

 
@tool
def call_score_jds(jd_indices):
    """
    Scores the user's CV against one or more job descriptions (JDs) to evaluate fit. You can also
    think of this as ranking JDs based on how well they match the CV.

    Args:
        jd_indices (List[str]): One or more job description indices to compare against the user's CV.

    """
    return

@tool
def call_synthesize_jds(jd_indices):
    """
     Summarizes and compares multiple job descriptions to extract insights about the job market, 
    key skill demands, or overlapping patterns across roles.

    Args:
        jd_indices (List[str]): The job description indices to synthesize.

    Returns:
        A narrative synthesis that describes:
        - Similarities and differences among the JDs
        - Common required skills, tools, or qualifications
        - Trends or focus areas across the roles
        - Any anomalies or outliers

    """
    return

@tool
def call_job_seacher(task_title):
    """
    Initiates a job search for a given field or role. This is the *first* tool you must call 
    when the user mentions a domain but hasn't provided any job indices yet.

    Args:
        task_title (str): The job title or field of interest. The more detail the better. Alway include number of 3 or more. For example: 3 job data analyst in marketing domain, ...
    """
    # This tool is not returning anything: we're just using it
    # as a way for LLM to signal that it needs to hand off to planner agent
    return


def jd_agent_node(state: AgentState):
    print('--- jds expert ---')
    llm = get_llm().bind_tools([call_score_jds, call_synthesize_jds, call_job_seacher])
    print('state:  ', state)
    response = llm.invoke([SystemMessage(JD_SYSTEM_PROMPT)] + state["messages"])
    print(response)
    if len(response.tool_calls) > 0:
        if response.tool_calls[0]['name'] == 'call_job_seacher':
            return Command(
				goto = 'job_seacher_agent',
                graph=Command.PARENT,
				update={"messages": [AIMessage(response.tool_calls[0]['args']['task_title'])],'sender': 'jd_agent'},
			)
        elif response.tool_calls[0]['name'] == 'call_score_jds':
            # return Command(
			# 	goto = 'score_jds',
			# 	update={"messages": [AIMessage(response.tool_calls.arg.task_title)],'sender': 'cv_jds_expert'},
			# )
            return Send(
                'score_jds', {"jd_indices": response.tool_calls[0]['args']['jd_indices'], 'cv': state['cv']}
            )
        elif response.tool_calls[0]['name'] == 'call_synthesize_jds':
            return Send(
                'synthesize_jds', {"jd_indices": response.tool_calls[0]['args']['jd_indices'], 'cv': state['cv']}
            )
            # return Command(
			# 	goto = 'synthesize_jds',
			# 	update={"messages": [AIMessage(response.tool_calls.arg.task_title)],'sender': 'cv_jds_expert'},
			# )
        else:
            pass
    return Command(
            # graph = Command.PARENT,
				update={"messages": [response],'sender': 'cv_jds_expert'},
			)



# def score_jds(jd_index):
#     """"""
#     return

# def synthesize_jds(jd_index):
#     """"""
#     return


builder = StateGraph(AgentState)
builder.add_node("assistant", jd_agent_node)
builder.add_node("tools", ToolNode([call_job_seacher,call_synthesize_jds,call_score_jds]))
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    tools_condition,
    # {'tools': 'tools',END: "router" }
)

builder.add_node("score_jds", score_agent)
builder.add_node("synthesize_jds", synthesize_agent)
JDExpert = builder.compile(checkpointer=memory)


In [33]:
# cv_jds_agent.invoke({'messages': 'hi', "jd_indices": ['1337', '4946']})

In [34]:
# cv_jds_agent.invoke({'messages': 'dạo này thị trường ngành content marketing như thế nào \no_think', "jd_indices": ['1337', '4946']}, config)


In [35]:
workflow = StateGraph(AgentState)
workflow.add_node("coordinator", coordinator_node)
workflow.add_node("job_searcher_agent", job_seacher)
workflow.add_node("router", router)
workflow.add_node("cv_agent", CVExpert)
workflow.add_node("jd_agent", JDExpert)


workflow.set_entry_point("coordinator")

from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()
career_agent = workflow.compile(checkpointer=memory)

In [36]:
# career_agent.invoke({'messages': 'hello \no_think', "jd_indices": ['1337', '4946']}, config)

In [37]:
# career_agent.invoke({'messages': 'bạn làm được gì \no_think', "jd_indices": ['1337', '4946']}, config)


In [26]:
# career_agent.invoke({'messages': 'tôi cần tìm 2 job data analyst \no_think', "jd_indices": ['1337', '4946']}, config)

state:------ {'messages': [HumanMessage(content='hello \no_think', additional_kwargs={}, response_metadata={}, id='a236c6e9-1d22-4749-b91d-621ffbc4c819'), AIMessage(content="Hello! I'm CareerFlow, your AI career coordinator. How can I assist you today?", additional_kwargs={}, response_metadata={}, id='1b28b692-8cc7-456d-ad98-3ec0fd042546'), HumanMessage(content='tôi cần tìm 2 job data analyst \no_think', additional_kwargs={}, response_metadata={}, id='cc34d8ec-53ac-413f-aff5-9b84f96df815')], 'jds': []}
res:------ next_step='job_searcher_agent' message_to_user='Được rồi, tôi sẽ giúp bạn tìm 2 công việc Data Analyst. Bạn có thể cung cấp thêm thông tin như vị trí làm việc (thuộc quốc gia hoặc thành phố nào), mức lương mong muốn, hay thời gian làm việc (full-time, part-time, remote) để tôi có thể lọc kết quả tốt hơn không?' message_to_next_agent='Người dùng cần tìm 2 công việc Data Analyst. Họ có thể cung cấp thêm thông tin về vị trí, mức lương, và hình thức làm việc.'
--job searcher--
--s

{'messages': [HumanMessage(content='hello \no_think', additional_kwargs={}, response_metadata={}, id='a236c6e9-1d22-4749-b91d-621ffbc4c819'),
  AIMessage(content="Hello! I'm CareerFlow, your AI career coordinator. How can I assist you today?", additional_kwargs={}, response_metadata={}, id='1b28b692-8cc7-456d-ad98-3ec0fd042546'),
  HumanMessage(content='tôi cần tìm 2 job data analyst \no_think', additional_kwargs={}, response_metadata={}, id='cc34d8ec-53ac-413f-aff5-9b84f96df815'),
  AIMessage(content='<think>\n\n</think>\n\nDưới đây là 2 công việc Data Analyst mà bạn có thể quan tâm:\n\n1. **Data Analyst - Senior Research Assistant**  \n   - [Xem chi tiết](http://jobs.hku.hk//cw/en/job/530224/data-analyst-at-the-rank-of-senior-research-assistant)  \n   - Hình thức làm việc: Fulltime  \n\n2. **Data Analyst - Senior Research Assistant**  \n   - [Xem chi tiết](http://jobs.hku.hk//cw/en/job/528791/data-analyst-at-the-rank-of-senior-research-assistant)  \n   - Hình thức làm việc: Fulltime 

In [27]:
# state= {'messages': [HumanMessage('align/rewrite my cv format')], 'cv': CV_CONTENT}

# for i, (msg, metadata) in enumerate(career_agent.stream(state, config, stream_mode="messages")):
#     print(metadata["langgraph_node"], i, msg.content, )

In [28]:
# state= {'messages': [HumanMessage('just do it')], 'cv': CV_CONTENT}

# for i, (msg, metadata) in enumerate(career_agent.stream(state, config, stream_mode="messages")):
#     print(metadata["langgraph_node"], i, msg.content, )

In [38]:
state= {'messages': [HumanMessage('chấm điểm job 4943 và 7376')], 'cv': CV_CONTENT}

for i, (msg, metadata) in enumerate(career_agent.stream(state, config, stream_mode="messages")):
    
    print(metadata["langgraph_node"], i, msg.content, )

state:------ {'messages': [HumanMessage(content='chấm điểm job 4943 và 7376', additional_kwargs={}, response_metadata={}, id='31f3fb3e-9c75-45f7-914e-3befa8b74d8d')], 'cv': ' AI Engineer\nA highly motivated student with a strong passion for AI, backed by a solid foundation in mathematics,\nstatistics and programming\nanhduc4hl2003@gmail.com   +84911581889     anhduc4work\nExperience\nAI Researcher Intern  •  AVT Consulting and Technology\nJan 2025 - Present \nResearched and explored pre-trained models for image processing. \nCollaborated with the IS team to integrate AI into the system for the MVP project.\nData Analyst Intern  •  VNPT AI\nJul 2024 - Sep 2024\nDesigned and implemented real-time dashboards using Power BI.\nAchieved 1st place in an internal Data Science competition for interns.\nEducation\nData Science in Economics University  •  National Economics University\nSep 2021 - May 2025 \nGPA: 3.40/4.00\nRelevant Courses:\nNatural Language Processing 1&2 (A) – Implemented & wor

In [42]:
career_agent.get_state(config).values #['messages'][2].content

{'messages': [HumanMessage(content='chấm điểm job 4943 và 7376', additional_kwargs={}, response_metadata={}, id='31f3fb3e-9c75-45f7-914e-3befa8b74d8d'),
  HumanMessage(content='Người dùng muốn chấm điểm CV của họ so với các công việc có mã 4943 và 7376.(user said: chấm điểm job 4943 và 7376 /no_think', additional_kwargs={}, response_metadata={}, id='1c48e9be-ad45-4ef3-bdc4-8910d9edbb18'),
  AIMessage(content="<think>\n\n</think>\n\n**Summary of HR Evaluation Analysis:**\n\n- **Patterns or Trends:** Both evaluations highlight a mismatch between the candidate's background in AI and data science and the job requirements, particularly in terms of job title relevance, required expertise, and education/certification. The candidate’s project work history is a moderate strength but not sufficient to offset other mismatches.\n\n- **Overall Candidate Fit Across All Jobs:** The overall fit scores are low (4 and 5), indicating that the candidate does not align well with either role. The strongest 

In [ ]:
# workflow = StateGraph(AgentState)
# workflow.add_node("coordinator", coordinator_node)
# workflow.set_entry_point("coordinator")

# workflow.add_node("job_searcher_agent", job_agent_node)




# workflow.add_node("cv_jd_agent", cv_jd_expert)
# workflow.add_node("format_reviewer", format_reviewer)
# workflow.add_node("content_reviewer", ContentReviewer)
# workflow.add_node("cv_writer", cv_writer)



# workflow.add_node("job_searcher_agent", job_seacher)
# workflow.add_node("router", router)
# workflow.add_node("cv_jd_agent", CVJDExpert)
# workflow.add_node("cv_jds_agent", CVJDsExpert)



# from langgraph.checkpoint.memory import MemorySaver
# memory = MemorySaver()
# career_agent = workflow.compile(checkpointer=memory)

In [10]:
from agent.workflow import career_agent

In [11]:
CV_CONTENT = """ AI Engineer
A highly motivated student with a strong passion for AI, backed by a solid foundation in mathematics,
statistics and programming
anhduc4hl2003@gmail.com   +84911581889     anhduc4work
Experience
AI Researcher Intern  •  AVT Consulting and Technology
Jan 2025 - Present 
Researched and explored pre-trained models for image processing. 
Collaborated with the IS team to integrate AI into the system for the MVP project.
Data Analyst Intern  •  VNPT AI
Jul 2024 - Sep 2024
Designed and implemented real-time dashboards using Power BI.
Achieved 1st place in an internal Data Science competition for interns.
Education
Data Science in Economics University  •  National Economics University
Sep 2021 - May 2025 
GPA: 3.40/4.00
Relevant Courses:
Natural Language Processing 1&2 (A) – Implemented & worked with transformer models, sentiment analysis, and
text classication.
Deep Learning (A) – Implemented neural networks, CNNs, and RNNs for real-world applications.
Big Data (A) – Applied distributed computing frameworks (Spark, Hadoop) for large-scale data processing.
Time Series Analysis (A+) – Forecasting techniques using ARIMA variants.
Physics Major  •  Ha Tinh High School for the Gifted
Sep 2018 - Jun 2021
Certication
IBM: AI Engineering Professional Certicate • Jan 2024
Image processing with OpenCV and Pillow, model building with TensorFlow, Keras, and PyTorch.
IBM: Data Science Professional Certicate • Sep 2023
Mastered Python, SQL, and Machine Learning through hands-on projects.
Others:  Top 10 H4TF Competition: Data Analysis - Unveil Data Power, Top 10FTU: Data Science Talent Competition, Fine
tuning Large Language Models, Building Systems with the ChatGPT API, Machine Learning in Production
(DeepLearning.AI), Data Engineer Professional Certicate, Generative AI for Data Scientists Specialization (IBM),
Introduction to LangGraph (LangChain), ...
Certication
Multi Agent Career Assistant
Developed a multi-agent system to assist job seekers in nding relevant jobs and optimizing their CVs.
Designed a job retrieval pipeline using semantic search with Atlas MongoDB and Nomic Embed Text.
Implemented agent coordination with LangGraph to automate job matching, CV evaluation, and CV adjustment.
Leveraged LLama 3.3 70B to provide personalized feedback and improvements for job applications.
Built a Gradio-basedUI for chat interaction, supporting CV uploads and automatic information extraction.
RAG System on PDF Documents
Developed an AI-powered Retrieval-Augmented Generation (RAG) system for querying and summarizing large PDF
documents.
Implemented document chunking to preprocess and split large PDFs into structured text.
Integrated Llama-based LLMs using LangChain to generate accurate and context-aware summaries.
Optimized retrieval mechanisms to improve response accuracy by 23% compared to LLM-only approach.
Impact of news on VN30 Index
Analyzed nancial news impact on VN30 stock index movements using NLP and ML models.
Collected and labeled 10,000+ nancial news articles from CafeF, VietStock, ...
Engineered textual sentiment scores as features for stock return predictions.
Built predictive models using LightGBM, ... achieving 62% accuracy on historical return data.
Tech stack
Programming Languages: Python, R
Databases: MySQL, PostgreSQL, MongoDB
Frameworks: Pytorch
Languages
Chinese - Basic
English - IELTS"""

In [12]:
state= {'messages': [HumanMessage('chấm điểm job 4943 và 7376')], 'cv': CV_CONTENT}
config = {"configurable": {"thread_id": "1"}}

for i, (msg, metadata) in enumerate(career_agent.stream(state, config, stream_mode="messages")):
    
    print(metadata["langgraph_node"], i, msg.content, )

state:------ {'messages': [HumanMessage(content='chấm điểm job 4943 và 7376', additional_kwargs={}, response_metadata={}, id='5028348e-ff73-4e2e-ae78-d74c3cfcf8e3')], 'cv': ' AI Engineer\nA highly motivated student with a strong passion for AI, backed by a solid foundation in mathematics,\nstatistics and programming\nanhduc4hl2003@gmail.com   +84911581889     anhduc4work\nExperience\nAI Researcher Intern  •  AVT Consulting and Technology\nJan 2025 - Present \nResearched and explored pre-trained models for image processing. \nCollaborated with the IS team to integrate AI into the system for the MVP project.\nData Analyst Intern  •  VNPT AI\nJul 2024 - Sep 2024\nDesigned and implemented real-time dashboards using Power BI.\nAchieved 1st place in an internal Data Science competition for interns.\nEducation\nData Science in Economics University  •  National Economics University\nSep 2021 - May 2025 \nGPA: 3.40/4.00\nRelevant Courses:\nNatural Language Processing 1&2 (A) – Implemented & wor

In [19]:
a = {}
a.update({'hi': ''})
a


{'hi': ''}

In [15]:
{}

{}

In [20]:
{'3'+ '3': '3'}

{'33': '3'}

In [21]:
'": "a'[2:]

' "a'